In [32]:
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
import random
from tensorflow.python.keras import Input, Model, Sequential
import nengo
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, \
    Conv3D, LSTM
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K

In [33]:
dataset_file = os.path.join('..', 'dataset_result', 'bci_dataset.npz')
dataset = np.load(dataset_file)

features, labels = dataset['features'], dataset['labels']
features = features.reshape((features.shape[0], 14, -1))

labels = labels.reshape((-1, 1))
labels = OneHotEncoder().fit_transform(labels).toarray()

# set seed to produce consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [34]:
def lstm_model():
    model = Sequential([
        LSTM(124, input_shape=(features.shape[1:]), activation=tf.nn.relu, return_sequences=True),
        Dropout(0.4),
        BatchNormalization(),
        LSTM(124, activation=tf.nn.relu),
        Dropout(0.3),
        BatchNormalization(),
        Dense(64, activation=tf.nn.relu),
        Dropout(0.2),
        Dense(2, activation=tf.nn.softmax, name='output_layer')
    ])

    return model

In [35]:
def run_network(model, train, valid, test, iteration, epochs=30):
    x_train, y_train = train[0], train[1]
    x_val, y_val = valid[0], valid[1]
    x_test, y_test = test[0], test[1]

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val),
              callbacks=[EarlyStopping(patience=8, verbose=1, restore_best_weights=True)]
              )
    eval = model.evaluate(x_test, y_test)
    print('{}. Accuracy: '.format(iteration), eval[1] * 100)
    return eval[1]

In [36]:
results = []

num_splits = 10
iteration = 1

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
for train_idx, val_idx in KFold(n_splits=num_splits).split(x_train):
    x_train_curr, y_train_curr = x_train[train_idx], y_train[train_idx]
    x_val, y_val = x_train[val_idx], y_train[val_idx]

    model = lstm_model()
    results.append(run_network(model, (x_train_curr, y_train_curr), (x_val, y_val), (x_test, y_test), iteration))
    K.clear_session()

    iteration += 1



Epoch 1/30
63/63 [==============================] - 13s 124ms/step - loss: 0.9451 - accuracy: 0.4778 - val_loss: 0.9269 - val_accuracy: 0.6027

In [37]:
print('RESULTS:')
for result in results:
    print(result)

print('Average:', np.average(results))
print('Max:', max(results))


RESULTS:
0.47983869910240173
0.4973118305206299
0.5255376100540161
0.4973118305206299
0.46639785170555115
0.5013440847396851
0.48521506786346436
0.5
0.5349462628364563
0.4865591526031494
Average: 0.4974462389945984
Max: 0.5349462628364563
